# Key Components and Transaction Flow


## Introduction to Hyperledger Fabric Architecture
### Arianna Groetsema


* [VIDEO](https://youtu.be/nyNUvtsmZNE)

> "Hyperledger Fabric is so unique, because it allows for modular consensus and membership service. This means that algorithms for consensus, identity verification are plug-and-play,resulting in a universal blockchain architecture, that can be applied to most industries or business models."<br><br>
"Channels are another unique feature. They allow transactions to be private between two actors, while still being verified and committed to the blockchain."

## Roles within a Hyperledger Fabric Network

There are three different types of roles within a Hyperledger Fabric network:

### Clients
Clients are applications that act on behalf of a person to propose transactions on the network.

### Peers
Peers maintain the state of the network and a copy of the ledger. There are two different types of peers: **endorsing** and **committing** peers. However, there is an overlap between endorsing and committing peers, in that **endorsing peers are a special kind of committing peers**. All peers commit blocks to the distributed ledger.
- Endorsers **simulate** and endorse transactions
- Committers **verify** endorsements and **validate** transaction results, **prior to committing transactions to the blockchain**.

### Ordering Service 
The ordering service accepts endorsed transactions, orders them into a block, and delivers the blocks to the committing peers.

## How to Reach Consensus

In a distributed ledger system, **consensus** is the process of reaching agreement on the next set of transactions to be added to the ledger. In Hyperledger Fabric, consensus is made up of three distinct steps:
* Transaction endorsement
* Ordering
* Validation and commitment.

These three steps ensure the policies of a network are upheld. We will explore how these steps are implemented by exploring the transaction flow.

## Transaction Flow (Step 1)

Within a Hyperledger Fabric network, transactions start out with client applications sending transaction proposals, or, in other words, proposing a transaction to endorsing peers.

![transaction-flow-fabric-1](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/21431955acd5b7888ca8d393c94deaf8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Key_Components_-_Transaction_Proposal.png)

**Client applications** are commonly referred to as **applications** or **clients**, and allow people to communicate with the blockchain network. Application developers can leverage the Hyperledger Fabric network through the application SDK.

## Transaction Flow (Step 2)

Each endorsing peer simulates the proposed transaction, without updating the ledger. The endorsing peers will capture the set of **Read** and **Written** data, called **RW Sets**. These RW sets capture what was read from the current world state while simulating the transaction, as well as what would have been written to the world state had the transaction been executed. These RW sets are then signed by the endorsing peer, and returned to the client application to be used in future steps of the transaction flow.

![transaction-flow-fabric-2](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/13e5a6a80c0e150f46d45ec0634b86b8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_flow_step_2.png)

### Endorsing peers must hold smart contracts in order to simulate the transaction proposals.